In [1]:
import pandas as pd
import numpy as np
import os

import snscrape.modules.twitter as sntwitter
from time import sleep

from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
import pickle

pd.set_option('Display.max_columns', 100)

In [2]:
def TWITTER_QUERY(**kwargs):
    from datetime import datetime
    # twitter 검색이 한글로 되어있는지 확인

    language_dict = {'모든 언어' : 'any', '구자라트어' : 'gu', '그리스어' : 'el', '네덜란드어' : 'nl', '노르웨이어' : 'no', '덴마크어' : 'da'
                    , '독일어' : 'de', '러시아어' : 'ru' , '루마니아어' : 'ro', '마라티어' : 'mr' , '바스크어' : 'eu', '베트남어' : 'vi'
                    , '벵골어' : 'bn', '불가리아어' : 'bg', '세르비아어' : 'sr', '스웨덴어' : 'sv', '스페인어' : 'es', '슬로바키아어' : 'sk'
                    , '아랍어' : 'ar', '아랍어(여성)' : 'ar-x-fm', '영어' : 'en', '우르두어' : 'ur', '우크라이나어' : 'uk', '이탈리아어' : 'it'
                    , '인도네시아어' : 'id', '일본어' : 'ja', '중국어(간체)' : 'zh-cn', '중국어(번체)' : 'zh-tw', '체코어' : 'cs', '카탈로니아어' : 'ca'
                    , '칸나다어' : 'kn', '크로아티아어' : 'hr', '타밀어' : 'ta', '태국어' : 'th', '터키어' : 'tr', '페르시아어' : 'fa', '포르투갈어' : 'pt'
                    , '폴란드어' : 'pl', '프랑스어' : 'fr', '핀란드어' : 'fi', '한국어' : 'ko', '헝가리어' : 'hu', '히브리어' : 'he', '힌디어' : 'hi'}

    """Query Settings"""
    # Query
    if kwargs['link_bool'] == False and kwargs['reply_bool'] == False:
        TOTAL_QUERY = ('{allword}' + ' "{exact}"' + ' ({any})' + ' -{exclude}' + ' (#{hashtag})' + ' lang:{lang}' \
                    + ' until:{until}' + ' since:{since}' + ' -filter:links' + ' -filter:replies')\
                        .format(allword = kwargs['allword_query'], exact = kwargs['exact_query'], any = kwargs['anyword_query'], exclude = kwargs['excludeword_query']
                        ,hashtag = kwargs['hashtag_query'], lang = language_dict[kwargs['selected_lang']], until = datetime.strftime(kwargs['end_date'], '%Y-%m-%d'),  since = datetime.strftime(kwargs['start_date'], '%Y-%m-%d'))

    elif kwargs['link_bool'] == False:
        TOTAL_QUERY = ('{allword}' + ' "{exact}"' + ' ({any})' + ' -{exclude}' + ' (#{hashtag})' + ' lang:{lang}' \
                    + ' until:{until}' + ' since:{since}' +  ' -filter:replies')\
                        .format(allword = kwargs['allword_query'], exact = kwargs['exact_query'], any = kwargs['anyword_query'], exclude = kwargs['excludeword_query']
                        ,hashtag = kwargs['hashtag_query'], lang = language_dict[kwargs['selected_lang']], until = datetime.strftime(kwargs['end_date'], '%Y-%m-%d'),  since = datetime.strftime(kwargs['start_date'], '%Y-%m-%d'))

    elif kwargs['reply_bool'] == False:
        TOTAL_QUERY = ('{allword}' + ' "{exact}"' + ' ({any})' + ' -{exclude}' + ' (#{hashtag})' + ' lang:{lang}' \
                    + ' until:{until}' + ' since:{since}' +  ' -filter:links')\
                        .format(allword = kwargs['allword_query'], exact = kwargs['exact_query'], any = kwargs['anyword_query'], exclude = kwargs['excludeword_query']
                        ,hashtag = kwargs['hashtag_query'], lang = language_dict[kwargs['selected_lang']], until = datetime.strftime(kwargs['end_date'], '%Y-%m-%d'),  since = datetime.strftime(kwargs['start_date'], '%Y-%m-%d'))

    else:
        TOTAL_QUERY = ('{allword}' + ' "{exact}"' + ' ({any})' + ' -{exclude}' + ' (#{hashtag})' + ' lang:{lang}' \
                    + ' until:{until}' + ' since:{since}')\
                        .format(allword = kwargs['allword_query'], exact = kwargs['exact_query'], any = kwargs['anyword_query'], exclude = kwargs['excludeword_query']
                        ,hashtag = kwargs['hashtag_query'], lang = language_dict[kwargs['selected_lang']], until = datetime.strftime(kwargs['end_date'], '%Y-%m-%d'),  since = datetime.strftime(kwargs['start_date'], '%Y-%m-%d'))

    return TOTAL_QUERY

In [3]:
def Scrapper(st_range, ed_range, SAVE_FOLDER, allword_query = '', exact_query = '', anyword_query = '', excludeword_query = '', hashtag_query = '', selected_lang = '영어', link_bool = False, reply_bool = False):
    from datetime import timedelta
    from dateutil.relativedelta import relativedelta
    import pickle
    from time import sleep

    """Scrap the data"""
    for st, ed in list(zip(st_range, ed_range)):
        
        """List for saving information"""
        tweets = []
        
    # ******************** QUERY SETTING *************************************
    
        """QUERY SETTINGS"""
        allword_query = allword_query
        exact_query = exact_query
        anyword_query = anyword_query
        excludeword_query = excludeword_query
        hashtag_query= hashtag_query
        selected_lang = selected_lang

        link_bool = link_bool # if False, not scraping the contents which contain the link
        reply_bool = reply_bool # if False, not scraping the contents which replied.


    # ******************** QUERY SETTING *************************************

        start_date = st
        end_date = ed

        query_info = {'allword_query' : allword_query, 'exact_query' : exact_query, 'anyword_query' : anyword_query, 'excludeword_query': excludeword_query, 'hashtag_query' : hashtag_query
                    , 'selected_lang' : selected_lang, 'link_bool': link_bool, 'reply_bool': reply_bool, 'start_date' : start_date, 'end_date' : end_date}

        TOTAL_QUERY = TWITTER_QUERY(**query_info) 

        for idx, tweet in enumerate(sntwitter.TwitterSearchScraper(TOTAL_QUERY).get_items()):

            # if len(tweets) == limit:
            #     break
            # else:
            #     tweets.append([tweet.date, tweet.user.username, tweet.user.displayname, tweet.user.description, tweet.user.created, tweet.user.followersCount, tweet.user.friendsCount
            #                     , tweet.user.location, tweet.lang, tweet.outlinks, tweet.content, ', '.join(tweet.hashtags), tweet.replyCount, tweet.retweetCount, tweet.likeCount])

            try:
                tweets.append([tweet.date, tweet.user.username, tweet.user.displayname, tweet.user.description, tweet.user.created, tweet.user.followersCount, tweet.user.friendsCount
                                    , tweet.user.location, tweet.lang, tweet.outlinks, tweet.content, ', '.join(tweet.hashtags), tweet.replyCount, tweet.retweetCount, tweet.likeCount])
            except TypeError:
                tweets.append([tweet.date, tweet.user.username, tweet.user.displayname, tweet.user.description, tweet.user.created, tweet.user.followersCount, tweet.user.friendsCount
                                    , tweet.user.location, tweet.lang, tweet.outlinks, tweet.content, tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount])


            if (idx+1) % 500 == 0:

                if (idx+1) % 1000 == 0:
                    print("The Scrapper is working....Up to {} tweets have been scraped.".format((idx+1)))
                print('Wait for a while....')
                sleep(2.5)


        tweet_df = pd.DataFrame(tweets, columns = ['Date', 'UserName', 'DisplayName', 'UserDescription', 'UserCreated', 'UserFollowersCount', 'UserFollowingCount'
                                        , 'UserLocation', 'Lang', 'Outlinks', 'Content', 'Hashtags', 'ReplyCount', 'RetweetCount', 'LikeCount'])

        

        print("---{start} and {end} have been finished.---".format(start = datetime.strftime(start_date, '%Y%m%d'), end = datetime.strftime(end_date, '%Y%m%d')))

        """Saving the data"""
        try:
            with open('./data_scrap/' + SAVE_FOLDER + '/' + datetime.strftime(start_date, '%y%m%d') + 'TO' + datetime.strftime(end_date, '%y%m%d') + '.pickle', 'wb') as f:
                pickle.dump(tweet_df, f, pickle.HIGHEST_PROTOCOL)

        except FileNotFoundError:
            os.makedirs('./data_scrap/' + SAVE_FOLDER + '/')
            with open('./data_scrap/' + SAVE_FOLDER + '/' + datetime.strftime(start_date, '%y%m%d') + 'TO' + datetime.strftime(end_date, '%y%m%d') + '.pickle', 'wb') as f:
                pickle.dump(tweet_df, f, pickle.HIGHEST_PROTOCOL)

In [6]:
st_date = datetime(2010, 1, 1)
ed_date = datetime(2016, 1, 1)

st_dates = list(pd.date_range(st_date, ed_date, freq = '7D')[:-1])
ed_dates = list(pd.date_range(st_date, ed_date, freq = '7D')[1:])

# ed_dates[-1] = datetime(2022, 1, 1)

In [ ]:
# covid-19이란 말은 
# ******************** DATE SETTING ************************************
from tqdm import tqdm


for st_range, ed_range in tqdm(zip(st_dates, ed_dates)):
    st_range = [st_range]
    ed_range = [ed_range]
    anyword_query = 'S&P500 OR SP500 OR MSCI OR DowJones OR $SPY'
    SAVE_FOLDER = 'twitter_example_2010-2015'


    Scrapper(st_range = st_range, ed_range = ed_range, SAVE_FOLDER = SAVE_FOLDER, anyword_query = anyword_query)

In [54]:
# lan en 이여도 아닌 것들이 잇다 (알파벳 잇는 애들)
# 내용은 content
# outlink link 글이 잇는지 없는지
# user loc : 완벽하게 나오지 않음
# 

In [3]:
Output_final = pd.concat([pd.read_pickle(r'../data_scrap/Final/'+x) for x in os.listdir('../data_scrap/Final/')])
Output_final.reset_index(drop = True, inplace = True)

In [4]:
Output_final.head()

,Date,UserName,DisplayName,UserDescription,UserCreated,UserFollowersCount,UserFollowingCount,UserLocation,Lang,Outlinks,Content,Hashtags,ReplyCount,RetweetCount,LikeCount
0,2010-01-07 22:36:37+00:00,willrobichau454,Will Robichaud,3B Mechanical Engineer special interests in Bu...,2010-01-07 19:46:42+00:00,8,8,Waterloo/Toronto ON,en,None,Figuring out MSCI 454 tweeting...,None,0,0,0
1,2010-01-07 22:30:11+00:00,wallstreet,wall street,Get Wall Street market news updates. Get finan...,2007-02-05 17:26:53+00:00,15671,47,"New York, NY",en,None,"US markets mixed, Dow finishes higher(up 33 po...",None,0,0,0
2,2010-01-07 22:29:49+00:00,Z_Alerts,Best Market Alerts,Uses Zignals Stock Alerts to bring the best to...,2009-08-13 21:15:22+00:00,281,20,Ireland,en,None,"20-day high alerts: $SPY, $DDM, $DIA, $XLI, $X...",None,0,0,0
3,2010-01-07 22:25:21+00:00,ianmerricks,Ian Merricks,,2009-02-23 16:35:37+00:00,66,445,London,en,None,"Well, my pension fund is now £95,000 = 38% sin...",None,0,0,0
4,2010-01-07 21:46:11+00:00,VistaWEB,VistaWEB,VistaWEB leader DIGITAL IPO Capital Formation ...,2009-07-02 09:29:43+00:00,988,1531,"ÜT: 40.32605,-73.98319",en,None,"Thu, January 7, 4pm close : S&P500 .....&gt;&g...",None,0,0,0


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

c:\Users\user\anaconda3\envs\sentiment\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [45]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [46]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
text = "Figuring out MSCI 454 tweeting..."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [48]:
scores

array([0.0451155 , 0.92581195, 0.0290726 ], dtype=float32)

In [49]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) Neutral 0.9258
2) Negative 0.0451
3) Positive 0.0291


In [69]:
Output_1 = Output_final[0:500000]
Output_2 = Output_final[500000:1000000].reset_index(drop=True)
Output_3 = Output_final[1000000:1500000].reset_index(drop=True)
Output_4 = Output_final[1500000:2000000].reset_index(drop=True)
Output_5 = Output_final[2000000:2119815].reset_index(drop=True)

In [71]:
from numpy import indices


for i in range(Output_5.shape[0]):
    text = Output_5.loc[i, 'Content']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    if (i+1) % 500 == 0:
        if (i+1) % 1000 == 0:
            print("It is working....Up to {} scores computed.".format((i+1)))
        print('Wait for a while....')

    # print(scores[1]*(0) + scores[0]*(-1) + scores[2]*(1))
    Output_5.loc[i,'Score'] = scores[0]*(-1) + scores[1]*(0) +  scores[2]*(1)
    
        
"""Saving the data"""

SAVE_FOLDER = 'bert'

with open('../data_scrap/' + SAVE_FOLDER + '/' + 'bert_5' + '.pickle', 'wb') as f:
    pickle.dump(Output_5, f, pickle.HIGHEST_PROTOCOL)

Wait for a while....
It is working....Up to 1000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 2000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 3000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 4000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 5000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 6000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 7000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 8000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 9000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 10000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up to 11000 scores computed.
Wait for a while....
Wait for a while....
It is working....Up 

In [43]:
with open('../data_scrap/bert/bert_1.pickle', 'wb') as f:
    pickle.dump(Output, f)

In [35]:
scores

array([0.00672401, 0.27609423, 0.71718174], dtype=float32)

In [28]:
Output

,Date,UserName,DisplayName,UserDescription,UserCreated,UserFollowersCount,UserFollowingCount,UserLocation,Lang,Outlinks,Content,Hashtags,ReplyCount,RetweetCount,LikeCount,Score
0,2010-01-07 22:36:37+00:00,willrobichau454,Will Robichaud,3B Mechanical Engineer special interests in Bu...,2010-01-07 19:46:42+00:00,8,8,Waterloo/Toronto ON,en,None,Figuring out MSCI 454 tweeting...,None,0,0,0,-0.016043
1,2010-01-07 22:30:11+00:00,wallstreet,wall street,Get Wall Street market news updates. Get finan...,2007-02-05 17:26:53+00:00,15671,47,"New York, NY",en,None,"US markets mixed, Dow finishes higher(up 33 po...",None,0,0,0,0.710458
2,2010-01-07 22:29:49+00:00,Z_Alerts,Best Market Alerts,Uses Zignals Stock Alerts to bring the best to...,2009-08-13 21:15:22+00:00,281,20,Ireland,en,None,"20-day high alerts: $SPY, $DDM, $DIA, $XLI, $X...",None,0,0,0,0.443475
3,2010-01-07 22:25:21+00:00,ianmerricks,Ian Merricks,,2009-02-23 16:35:37+00:00,66,445,London,en,None,"Well, my pension fund is now £95,000 = 38% sin...",None,0,0,0,-0.384274
4,2010-01-07 21:46:11+00:00,VistaWEB,VistaWEB,VistaWEB leader DIGITAL IPO Capital Formation ...,2009-07-02 09:29:43+00:00,988,1531,"ÜT: 40.32605,-73.98319",en,None,"Thu, January 7, 4pm close : S&P500 .....&gt;&g...",None,0,0,0,0.107409
5,2010-01-07 21:15:02+00:00,marketclose,Shaun McCarthy,"A Twitter API App in C# - Personal Project, no...",2009-04-08 20:55:55+00:00,17,2,New York,en,None,"Market Close for 1/7/2010 - Dow: 10,606.86 (+0...",None,0,0,0,0.102674
6,2010-01-07 21:08:20+00:00,ChartingStock,ChartingStock,"Technical Analysis of Stocks, Commodities & Cu...",2009-01-03 21:45:24+00:00,4988,89,Global,en,None,$SPY Institutional Sales=$113.9282 $113.8018 $...,Mkt,0,0,0,-0.061401
7,2010-01-07 20:54:03+00:00,BostonRoca,BostonSoldierOfGod💯,Living In An Upside Down World!❤️Reading the B...,2009-09-17 22:55:41+00:00,1139,1282,"Boston, MA",en,None,$SPY puts influx now,None,0,0,0,0.140013
8,2010-01-07 20:46:18+00:00,Trending123,John Lansing,Technical analysis trader with a passion for n...,2008-07-24 12:44:54+00:00,2916,57,Oklahoma,en,None,SP500 Futures gunning for a close on the highs...,None,0,0,0,0.236187
9,2010-01-07 20:33:32+00:00,TheOptionMentor,Randy Burris,Teaching how to trade Option Spreads for Month...,2009-11-28 04:17:10+00:00,45,39,,en,None,A Strangle on the $SPY might be effective here...,None,0,0,0,0.097992


In [ ]:
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
folder='../data_scrap/twitter_example_2010-2015/'

for filename in os.listdir(folder):
    if filename.endswith('.pickle'):
        with open(folder + filename, 'rb') as f:
            data = pickle.load(f)
            

In [ ]:
import os
import pickle

folder='../data_scrap/twitter_example_2010-2015/'
db = {}  
for filename in os.listdir(folder):  
    if filename.endswith('.pickle'):
        myfile = open(folder+filename,"rb")
        db[os.path.splitext(filename)[0]]= pickle.load(myfile)
        myfile.close()
        print(filename)

print(db)
# myfile = open("merge/merge.pickle","wb")
# pickle.dump(db, myfile)
# myfile.close()

In [ ]:
import os
import pickle

folder='../data_scrap/twitter_example_2010-2015/'
db = {}  
for filename in os.listdir(folder):  
    if filename.endswith('.pickle'):
        d = pickle.load(os.path.join())
        myfile.close()
        print(filename)



In [21]:
db.items()

dict_items([('100101TO100108',                          Date         UserName           DisplayName  \
0   2010-01-07 22:36:37+00:00  willrobichau454        Will Robichaud   
1   2010-01-07 22:30:11+00:00       wallstreet           wall street   
2   2010-01-07 22:29:49+00:00         Z_Alerts    Best Market Alerts   
3   2010-01-07 22:25:21+00:00      ianmerricks          Ian Merricks   
4   2010-01-07 21:46:11+00:00         VistaWEB              VistaWEB   
..                        ...              ...                   ...   
197 2010-01-01 17:43:46+00:00         tecktrdr                 G-TBO   
198 2010-01-01 14:30:10+00:00     BoardCentral          BoardCentral   
199 2010-01-01 08:21:18+00:00      brokerz_spx  Brokerz S&P 500 news   
200 2010-01-01 02:16:17+00:00       Eagledives          Lance Carson   
201 2010-01-01 01:07:33+00:00         alienone              alienone   

                                       UserDescription  \
0    3B Mechanical Engineer special interests 

In [36]:


a = pd.DataFrame(list(db.items()), columns = ['column1', 'column2'])['column2'][0]
b = pd.DataFrame(list(db.items()), columns = ['column1', 'column2'])['column2'][1]

pd.concat([a,b], axis = 0)

,Date,UserName,DisplayName,UserDescription,UserCreated,UserFollowersCount,UserFollowingCount,UserLocation,Lang,Outlinks,Content,Hashtags,ReplyCount,RetweetCount,LikeCount
0,2010-01-07 22:36:37+00:00,willrobichau454,Will Robichaud,3B Mechanical Engineer special interests in Bu...,2010-01-07 19:46:42+00:00,8,8,Waterloo/Toronto ON,en,None,Figuring out MSCI 454 tweeting...,None,0,0,0
1,2010-01-07 22:30:11+00:00,wallstreet,wall street,Get Wall Street market news updates. Get finan...,2007-02-05 17:26:53+00:00,15671,47,"New York, NY",en,None,"US markets mixed, Dow finishes higher(up 33 po...",None,0,0,0
2,2010-01-07 22:29:49+00:00,Z_Alerts,Best Market Alerts,Uses Zignals Stock Alerts to bring the best to...,2009-08-13 21:15:22+00:00,281,20,Ireland,en,None,"20-day high alerts: $SPY, $DDM, $DIA, $XLI, $X...",None,0,0,0
3,2010-01-07 22:25:21+00:00,ianmerricks,Ian Merricks,,2009-02-23 16:35:37+00:00,66,445,London,en,None,"Well, my pension fund is now £95,000 = 38% sin...",None,0,0,0
4,2010-01-07 21:46:11+00:00,VistaWEB,VistaWEB,VistaWEB leader DIGITAL IPO Capital Formation ...,2009-07-02 09:29:43+00:00,988,1531,"ÜT: 40.32605,-73.98319",en,None,"Thu, January 7, 4pm close : S&P500 .....&gt;&g...",None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2010-01-08 08:49:16+00:00,TradingFloorcom,TradingFloor,TradingFloor is a community for traders and in...,2009-02-06 21:42:08+00:00,22851,2717,,en,None,S&P500 0/+ Buy on dips towards 1139 targeting ...,None,0,0,0
221,2010-01-08 00:41:53+00:00,SBroker,SBroker,"Daily updates of SG market calls, based on off...",2009-02-05 01:38:01+00:00,156,7,"Raffles Place, Singapore",en,None,"Dow_Futures* 10552.00 +7.00 +0.07%, \nNASDAQ_F...",None,0,0,0
222,2010-01-08 00:40:51+00:00,SBroker,SBroker,"Daily updates of SG market calls, based on off...",2009-02-05 01:38:01+00:00,156,7,"Raffles Place, Singapore",en,None,"Dow 10606.86 +33.18 +0.31%, NASDAQ 2300.05 -1....",None,0,0,0
223,2010-01-08 00:30:53+00:00,Psando,Psando,,2008-10-14 20:06:48+00:00,350,291,,en,None,Market getting a bit over extended...but who a...,None,0,0,0


In [38]:
a.shape, b.shape

((202, 15), (225, 15))